In [1]:
pip install langchain-community

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
from pinecone import Pinecone, ServerlessSpec 
import pinecone

e:\maniconda\envs\mchat\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:

PINECONE_API_KEY = "cde0135b-6b53-4821-a40a-285e9e324347"
PINECONE_API_ENV = "us-east-1"
PINECONE_INDEX_NAME = "medicine"


In [4]:
# Set the Pinecone API key as an environment variable
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [5]:
from langchain_pinecone import PineconeVectorStore

In [6]:
pc = Pinecone(
    api_key=os.environ.get("PINECONE_API_KEY")
)

# Check if the index exists, and create it if it doesn't
if PINECONE_INDEX_NAME not in [index.name for index in pc.list_indexes().index_list["indexes"]]:
    pc.create_index(
        name=PINECONE_INDEX_NAME, 
        dimension=384,  # Replace with the actual dimension of your embeddings
        metric='cosine',  # Or the appropriate metric for your use case
        spec=ServerlessSpec(
            cloud='aws',
            region=PINECONE_API_ENV
        )
    )

# Extract data from the PDF

In [7]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()
    

    return documents

In [8]:


extracted_data = load_pdf("data/")

In [9]:
# Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("Length of my chunk:", len(text_chunks))

# Download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hugging_face_embeddings()

Length of my chunk: 7020


e:\maniconda\envs\mchat\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [11]:

query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [12]:
from langchain_pinecone import PineconeVectorStore

In [13]:
# Create or load PineconeVectorStore
try:
    vector_store = PineconeVectorStore.from_existing_index(PINECONE_INDEX_NAME, embeddings)
except Exception as e:
    print(f"Error loading existing index: {e}")
    vector_store = PineconeVectorStore.from_texts(
        [chunk.page_content for chunk in text_chunks],
        embeddings,
        index_name=PINECONE_INDEX_NAME
    )

# Create PineconeVectorStore from texts
vector_store = PineconeVectorStore.from_texts(
    [chunk.page_content for chunk in text_chunks],
    embeddings,
    index_name=PINECONE_INDEX_NAME
)

In [14]:
vector_store = PineconeVectorStore.from_existing_index(PINECONE_INDEX_NAME, embeddings)

In [15]:
vector_store

In [16]:
# Perform a similarity search
query = "What are Allergies"
docs = vector_store.similarity_search(query, k=3)
print("Result:", docs)


Result: [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commonly c

In [17]:
# Perform a similarity search
query = "What are Heart Deases"

new = vector_store.similarity_search(query, k=3)
print("Result:", new)

Result: [Document(page_content='(disordered heart beat).'), Document(page_content='some cases, removal of a small portion of the heart (abla-\ntion), implantation of a pacemaker or a cardioverterdefibrillator, or maze surgery is needed.\nIf the heart rate cannot be quickly controlled, electri-\ncal cardioversion may be used. Cardioversion, the elec-tric shock to the chest wall, is usually performed emer-gencies. This device briefly suspends the heart’s activityand allows it to return to a normal rhythm.\nAblation destroys the heart tissue that causes the'), Document(page_content='threatening ventricular arrhythmias by recognizing themand then restoring a normal heart rhythm by pacing theheart or giving it an electric shock. They are implantedwithin the chest wall without major surgery and storeinformation for future evaluation by physicians. Auto-matic implantable defibrillators have proven to be moreeffective in saving lives than drugs alone. They often areused in conjunction with dru

In [18]:
# Define the prompt template
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}


Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [19]:
PROMPT =PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

In [20]:
# Initialize the language model
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [21]:
# Set up RetrievalQA

qa =RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_store.as_retriever(search_kwargs={'k': 2}),
        return_source_documents=True,
        chain_type_kwargs=chain_type_kwargs
    )

    
    

In [22]:
while True:
    user_input = input("Input Prompt: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting the chatbot.")
        break
    result = qa.invoke({"query": user_input})
    print("Response: ", result["result"])  
    



Response:  Diabetes is a metabolic disease caused by a deficiency of insulin, which is essential to process carbohydrates in the body. It can damage or cause failure of various body organs if left untreated.
Exiting the chatbot.
